# 平均報酬強化学習

参考：
* [Model-free Reinforcement Learning in Infinite-horizon Average-reward Markov Decision Processes](https://arxiv.org/abs/1910.07072)

今回は平均報酬強化学習におけるモデルフリーな探索を見てみます．Ergodicを考えます．

## Ergodicな場合

任意の方策について，それに誘導されるマルコフ連鎖がirreducible（transientが存在しない）かつaperiodic（periodが1）ならば，そのMDPはErgodicといいます．
**ここでは，ErgodicなMDPを多腕バンディット問題とみなして，Optimistic Mirror Descentで解く方法**を紹介します．これは$\sqrt{T}$リグレットを達成できます．

表記：
* 方策の定常分布：$\left(\mu^\pi\right)^{\top} P^\pi=\left(\mu^\pi\right)^{\top}$．つまり，定常分布から一回遷移しても，また定常分布に戻ります．
  * これは[RL_AverageReward.ipynb](RL_AverageReward.ipynb)でちょっと言及してます．特にpositive recurrent irreducibleならユニークな解を持ちます．簡単に言えば，**Ergodicなマルコフ連鎖はユニークな解を持ちます．**
* ErgodicなMDPでは期待収益について$J^\pi=\left(\mu^\pi\right)^{\top} r^\pi$が成り立ち，これは状態に依存しません．
* ErgodicなMDPでは，次のベルマン方程式の解$q^\pi$が存在し，それは定数項についてユニークです（$v^\pi(s)=\sum_a \pi(a \mid s) q^\pi(s, a)$）：
$$
J^\pi+q^\pi(s, a)=r(s, a)+\mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)}\left[v^\pi\left(s^{\prime}\right)\right]
$$
  * 普通の割引報酬では$Q^\pi(s, a)=r(s, a)+\mathbb{E}_{s^{\prime} \sim p(\cdot \mid s, a)}\left[V^\pi\left(s^{\prime}\right)\right]$ですね．平均報酬の$J^\pi+q^\pi(s, a)$が$Q^\pi(s, a)$に相当します．

特に$\sum_s \mu^\pi(s) v^\pi(s)=0$の制約を課せば，$q^\pi$は唯一の解を持ちます．このとき，
$$
v^\pi(s)=\sum_{t=0}^{\infty}\left(\mathbf{e}_s^{\top}\left(P^\pi\right)^t-\left(\mu^\pi\right)^{\top}\right) r^\pi
$$
を満たします．

---

**Mixing timeとHitting time**

次をergodic MDPのMixing timeといいます：

$$
t_{\text {mix }}:=\max _\pi \min \left\{t \geq 1 \left\lvert\,\left\|\left(P^\pi\right)^t(s, \cdot)-\mu^\pi\right\|_1 \leq \frac{1}{4}\right., \forall s\right\}
$$

また，次をHitting timeといいます：

$$
t_{\text {hit }}:=\max _\pi \max _s \frac{1}{\mu^\pi(s)}
$$

**Distribution mismatch 係数**

次のことです．

$$
\rho:=\max _\pi \sum_s \frac{\mu^*(s)}{\mu^\pi(s)}
$$

すなわち，最適な定常分布から最も適当な方策によって生成される分布がこれを与えます．
明らかに，
$$
\rho \leq t_{\text {hit }} \sum_s \mu^*(s)=t_{\text {hit }}
$$
が成り立ちます．

今回の導出ではホライゾン$T$がmixing timeとhitting timeより十分大きく，どちらも$T/4$以下である状況を考えます．
また，**どちらも既知**であるとします．


### アルゴリズム

Optimistic Mirror Descentを使います

1. エピソードの長さ：$B=16 t_{\text {mix }} t_{\text {hit }}\left(\log _2 T\right)^2$， エピソードの数：$K = T / B$
2. for $k = 1, 2, \dots, K$
3. for $t = (k-1) B + 1, \dots, kB$：軌跡：$\mathcal{T}_k = \left(s_{(k-1) B+1}, a_{(k-1) B+1}, \ldots, s_{k B}, a_{k B}\right)$
4. for $s \in \mathcal{S}$：
  * $\widehat{\beta}_k(s, \cdot)=\operatorname{ESTIMATEQ}\left(\mathcal{T}_k, \pi_k, s\right)$
  * $\left(\pi_{k+1}^{\prime}(\cdot \mid s), \pi_{k+1}(\cdot \mid s)\right)= \operatorname{OoMdUPDATE}\left(\pi_k^{\prime}(\cdot \mid s), \widehat{\beta}_k(s, \cdot)\right)$

---

ここで，$\operatorname{ESTIMATEQ}\left(\mathcal{T}_k, \pi_k, s\right)$は次の処理をします：
基本的には重点サンプリングで状態$s$で各$a$を選択した際の期待リターンを計算してます．

* 初期値：$N=4 t_{\operatorname{mix}} \log _2 T( \text{window length minus 1})$，$\mathcal{T} = \left(s_{t_1}, a_{t_1}, \ldots, s_{t_2}, a_{t_2}\right)$
* $\tau \leftarrow t_1, i \leftarrow 0$

1. while $\tau \leq t_2 - N$
2. もし$s_\tau = s$なら，
  * $i \leftarrow i + 1$
  * Let $R=\sum_{t=\tau}^{\tau+N} r\left(s_t, a_t\right)$.
  * Let $y_i(a)=\frac{R}{\pi(a \mid s)} \mathbf{1}\left[a_\tau=a\right], \forall a . \quad\left(y_i \in \mathbb{R}^A\right)$ $\tau \leftarrow \tau+2 N$
  * $\tau \leftarrow \tau + 2N$
3. そうでないなら$\tau \leftarrow \tau + 1$
4. もし$i \neq 0$なら，$\frac{1}{i} \sum_{j=1}^i y_j$を返す
5. そうでないなら，$0$を返す．

得られた$\beta$は$\beta^{\pi_k}(s, \cdot):=q^{\pi_k}(s, \cdot)+N J^{\pi_k}$を無視できるバイアス項まで推定します．
$NJ^{\pi_k}$は全ての行動で同じなので，バイアス項$q^{\pi_k}(s, \cdot)$に従えば一番良いアームが選択できます．

---

得られた期待リターンを使って，$OOMDUPDATE$は多腕バンディット問題を解きます:
$$
\begin{aligned}
& \pi_{\text {next }}^{\prime}=\underset{\pi \in \Delta_A}{\operatorname{argmax}}\left\{\langle\pi, \widehat{\beta}\rangle-D_\psi\left(\pi, \pi^{\prime}\right)\right\} \\
& \pi_{\text {next }}=\underset{\pi \in \Delta_A}{\operatorname{argmax}}\left\{\langle\pi, \widehat{\beta}\rangle-D_\psi\left(\pi, \pi_{\text {next }}^{\prime}\right)\right\}
\end{aligned}
$$



### リグレットの解析

まずはリグレットを分解しましょう（そもそも平均報酬のリグレットの定義に注意しましょう）：

$$
\begin{aligned}
& R_T=\sum_{t=1}^T J^*-r\left(s_t, a_t\right) \\
& =B \sum_{k=1}^K\left(J^*-J^{\pi_k}\right)+\sum_{k=1}^K \sum_{t \in \mathcal{I}_k}\left(J^{\pi_k}-r\left(s_t, a_t\right)\right)
\end{aligned}
$$

$\mathcal{I}_k:=\{(k-1) B+1, \ldots, k B\}$はエピソード$k$中のタイムステップです．
平均報酬のPerformance difference lemmaを使うと，最初の項は

$$
B \sum_s \mu^*(s)\left[\sum_{k=1}^K \sum_a\left(\pi^*(a \mid s)-\pi_k(a \mid s)\right) q^{\pi_k}(s, a)\right]
$$

と同じです．ESTIMATEQの精度が高ければ，これは実質多腕バンディットを解いてるのと同じですね．
実際，次のようにESTIMATEQの精度はかなり高くなります：

$$
\begin{aligned}
& \left|\mathbb{E}_k\left[\widehat{\beta}_k(s, a)\right]-\beta^{\pi_k}(s, a)\right| \leq \mathcal{O}\left(\frac{1}{T}\right) \\
& \mathbb{E}_k\left[\left(\widehat{\beta}_k(s, a)-\beta^{\pi_k}(s, a)\right)^2\right] \leq \mathcal{O}\left(\frac{N^3 \log T}{B \pi_k(a \mid s) \mu^{\pi_k}(s)}\right)
\end{aligned}
$$

**ESTIMATEQの精度の証明**

これはエピソディックなケースと比べて結構厄介です．
エピソディックなケースでは
* エピソード１のリターン
* エピソード２のリターン
* ...

を独立して推定できますが，今回はエピソードのリセットがありません．なので，例えば
* エピソード中の$1, \dots, N$までのリターン
* エピソード中の$N, \dots, 2N$までのリターン
* ...

みたいにリターンを有限ステップで近似すると，１つ目と２つ目の推定量が独立しません．
これが厄介な点です．

以降，固定された$k$エピソード目と状態$s$だけ考えます．
* $k$エピソード内で，状態$s$を$M$回見つけたとします．
* $\widehat{\beta}_{k, i}(s, \cdot)$：$i$番目のインターバルでのリターンの推定量．とします．つまり，ESTIMATEQ中の$y_i$のことです．
* アルゴリズムより，

$$
\widehat{\beta}_k(s, a)= \begin{cases}\frac{\sum_{i=1}^M \widehat{\beta}_{k, i}(s, a)}{M} & \text { if } M>0 \\ 0 & \text { if } M=0\end{cases}
$$

が成立してます．ここで，$\widehat{\beta}_{k, i}(s, a)$は時刻$\tau_i \leq kB-N$から始まる長さ$N+1$の軌跡で構築されているので，

$$
\begin{aligned}
& \mathbb{E}\left[\widehat{\beta}_{k, i}(s, a) \mid s_{\tau_i}=s\right] \\
& =\operatorname{Pr}\left[a_{\tau_i}=a \mid s_{\tau_i}=s\right] \times \frac{r(s, a)+\mathbb{E}\left[\sum_{t=\tau_i+1}^{\tau_i+N} r\left(s_t, a_t\right) \mid\left(s_{\tau_i}, a_{\tau_i}\right)=(s, a)\right]}{\pi(a \mid s)} \\
& =r(s, a)+\sum_{s^{\prime}} p\left(s^{\prime} \mid s, a\right) \mathbb{E}\left[\sum_{t=\tau_i+1}^{\tau_i+N} r\left(s_t, a_t\right) \mid s_{\tau_i+1}=s^{\prime}\right] \\
& =r(s, a)+\sum_{s^{\prime}} p\left(s^{\prime} \mid s, a\right) \sum_{j=0}^{N-1} \mathbf{e}_{s^{\prime}}^{\top}\left(P^\pi\right)^j r^\pi \\
& =r(s, a)+\sum_{s^{\prime}} p\left(s^{\prime} \mid s, a\right) \sum_{j=0}^{N-1}\left(\mathbf{e}_{s^{\prime}}^{\top}\left(P^\pi\right)^j-\left(\mu^\pi\right)^{\top}\right) r^\pi+N J^\pi \\
& =r(s, a)+\sum_{s^{\prime}} p\left(s^{\prime} \mid s, a\right) v^\pi\left(s^{\prime}\right)+N J^\pi-\sum_{s^{\prime}} p\left(s^{\prime} \mid s, a\right) \sum_{j=N}^{\infty}\left(\mathbf{e}_{s^{\prime}}^{\top}\left(P^\pi\right)^j-\left(\mu^\pi\right)^{\top}\right) r^\pi \\
& =q^\pi(s, a)+N J^\pi-\delta(s, a) \\
& =\beta^\pi(s, a)-\delta(s, a)
\end{aligned}
$$

ここで，
$$
\delta(s, a) \triangleq \sum_{s^{\prime}} p\left(s^{\prime} \mid s, a\right) \sum_{j=N}^{\infty}\left(\mathbf{e}_{s^{\prime}}^{\top}\left(P^\pi\right)^j-\left(\mu^\pi\right)^{\top}\right) r^\pi
$$
としました．
[RL_useful_lemma/RL_AverageReward_lemma.ipynb](RL_useful_lemma/RL_AverageReward_lemma.ipynb)から，mixing timeを使うと，

$$
|\delta(s, a)| \leq \frac{1}{T^3}
$$
であり，
$$
\left|\mathbb{E}\left[\widehat{\beta}_{k, i}(s, a) \mid s_{\tau_i}=s\right]-\beta^\pi(s, a)\right| \leq \frac{1}{T^3}
$$
が成り立ちます．

よって$\hat{\beta}_{k, i}$は$\beta_k$の**ほぼ**unbiasedな推定値になってます．しかし，ここで注意すべきは，**$M$は確率変数であり，$\sum_{i=1}^M \widehat{\beta}_{k, i}(s, a)$と独立ではありません．**

$\mathbb{E}\left[\widehat{\beta}_k(s, a)\right]$について確認しましょう．
これと$\beta_k$の関係を調べますが，次の２ステップに分解します：

1. まず，$\widehat{\beta}_k(s, a)$が$\beta^\pi(s, a)$の**ほぼ**unbiasedな推定値になるようなimaginary worldを構築します
2. 続いて，imaginary worldにおける$\widehat{\beta}_k(s, a)$と真の世界での$\widehat{\beta}_k(s, a)$が近いことを示します．

---

**Imaginary world**

Imaginary worldとして，次の図を考えます：

![imagine](figs/Imaginary-world.png)

すなわち，$\tau_1 + 2N$ステップごとに，状態の分布が定常分布にリセットされる状況です．
* $w_i=\tau_i-\left(\tau_{i-1}+2 N\right)$ for $i>1$
* $w_1=\tau_1-((k-1) B+1)$

とします．どちらも，状態$s$を踏む前のインターバルを表します．
すると，
1. $M$は$w_1, w_2,\cdots$によって決定されます
2. $w_1$は$s_{(k-1) B+1}$と$P^\pi$のみによって決定され，$w_i$は$\mu^\pi$と$P^\pi$のみで決定されます（リセットのせいで）

よって，$\widehat{\beta}_{k, 1}(s, a), \widehat{\beta}_{k, 2}(s, a), \ldots$は$w_1, w_2, \dots$と$M$に対して独立してます（$\beta$とインターバルは関係ないので）．

後は独立性を使えば，$\hat{\beta}_{k,i}$と同じようにして
$$
\left|\mathbb{E}^{\prime}\left[\widehat{\beta}_k(s, a)\right]-\beta^\pi(s, a)\right|=\mathcal{O}\left(\frac{1}{T^2}\right)
$$
が示せます．

---

**Real world**

$\widehat{\beta}_k(s, a)$は
$$
X=\left(M, \tau_1, \mathcal{T}_1, \tau_2, \mathcal{T}_2, \ldots, \tau_M, \mathcal{T}_M\right)
$$
に対して決定的な関数であることに注意しましょう．ここで，$\mathcal{T}_i=\left(a_{\tau_i}, s_{\tau_i+1}, a_{\tau_i+1}, \ldots, s_{\tau_i+N}, a_{\tau_i+N}\right)$です．$\widehat{\beta}_k(s, a)=f(X)$と書くことにします．

以降では，
$$
\frac{\mathbb{E}\left[\widehat{\beta}_k(s, a)\right]}{\mathbb{E}^{\prime}\left[\widehat{\beta}_k(s, a)\right]}=\frac{\sum_X f(X) \mathbb{P}(X)}{\sum_X f(X) \mathbb{P}^{\prime}(X)} \leq \max _X \frac{\mathbb{P}(X)}{\mathbb{P}^{\prime}(X)}
$$
を示します．ここで，$\mathbb{P}$と$\mathbb{P}'$はそれぞれreal worldとimaginary worldでの確率測度です．
ここで，

$$
\begin{aligned}
&\begin{aligned}
& \mathbb{P}(X)=\mathbb{P}\left(\tau_1\right) \times \mathbb{P}\left(\mathcal{T}_1 \mid \tau_1\right) \times \mathbb{P}\left(\tau_2 \mid \tau_1, \mathcal{T}_1\right) \times \mathbb{P}\left(\mathcal{T}_2 \mid \tau_2\right) \times \cdots \times \mathbb{P}\left(\tau_M \mid \tau_{M-1}, \mathcal{T}_{M-1}\right) \\
& \times \mathbb{P}\left(\mathcal{T}_M \mid \tau_M\right) \times \operatorname{Pr}\left[s_t \neq s, \forall t \in\left[\tau_M+2 N, k B-N\right] \mid \tau_M, \mathcal{T}_M\right]
\end{aligned}\\
&\mathbb{P}(X)=\mathbb{P}\left(\tau_1\right) \times \mathbb{P}\left(\mathcal{T}_1 \mid \tau_1\right) \times \mathbb{P}\left(\tau_2 \mid \tau_1, \mathcal{T}_1\right) \times \mathbb{P}\left(\mathcal{T}_2 \mid \tau_2\right) \times \cdots \times \mathbb{P}\left(\tau_M \mid \tau_{M-1}, \mathcal{T}_{M-1}\right)\\
&\times \mathbb{P}\left(\mathcal{T}_M \mid \tau_M\right) \times \operatorname{Pr}\left[s_t \neq s, \forall t \in\left[\tau_M+2 N, k B-N\right] \mid \tau_M, \mathcal{T}_M\right]
\end{aligned}
$$
および
$$
\begin{aligned}
& \mathbb{P}^{\prime}(X)=\mathbb{P}\left(\tau_1\right) \times \mathbb{P}\left(\mathcal{T}_1 \mid \tau_1\right) \times \mathbb{P}^{\prime}\left(\tau_2 \mid \tau_1, \mathcal{T}_1\right) \times \mathbb{P}\left(\mathcal{T}_2 \mid \tau_2\right) \times \cdots \times \mathbb{P}^{\prime}\left(\tau_M \mid \tau_{M-1}, \mathcal{T}_{M-1}\right) \\
& \times \mathbb{P}\left(\mathcal{T}_M \mid \tau_M\right) \times \operatorname{Pr}\left[s_t \neq s, \forall t \in\left[\tau_M+2 N, k B-N\right] \tau_M, \mathcal{T}_M\right.
\end{aligned}
$$

なので，それぞれの違いはresetによる$\mathbb{P}\left(\tau_{i+1} \mid \tau_i, \mathcal{T}_i\right) \neq \mathbb{P}^{\prime}\left(\tau_{i+1} \mid \tau_i, \mathcal{T}_i\right)$
だけです．
ここで，

$$
\begin{aligned}
\mathbb{P}\left(\tau_{i+1} \mid \tau_i, \mathcal{T}_i\right) & =\sum_{s^{\prime} \neq s} \mathbb{P}\left(s_{\tau_i+2 N}=s^{\prime} \mid \tau_i, \mathcal{T}_i\right) \times \operatorname{Pr}\left[s_t \neq s, \forall t \in\left[\tau_i+2 N, \tau_{i+1}-1\right], s_{\tau_{i+1}}=s \mid s_{\tau_i+2 N}=s^{\prime}\right] \\
\mathbb{P}^{\prime}\left(\tau_{i+1} \mid \tau_i, \mathcal{T}_i\right) & =\sum_{s^{\prime} \neq s} \mathbb{P}^{\prime}\left(s_{\tau_i+2 N}=s^{\prime} \mid \tau_i, \mathcal{T}_i\right) \times \operatorname{Pr}\left[s_t \neq s, \forall t \in\left[\tau_i+2 N, \tau_{i+1}-1\right], s_{\tau_{i+1}}=s \mid s_{\tau_i+2 N}=s^{\prime}\right]
\end{aligned}
$$

であることに注意しましょう．Imaginary worldでは
$$\mathbb{P}^{\prime}\left(s_{\tau_i+2 N}=s^{\prime} \mid \tau_i, \mathcal{T}_i\right)=\mu^\pi\left(s^{\prime}\right) \mathrm{f}$$
なので，
$$
\frac{\mathbb{P}\left(s_{\tau_i+2 N}=s^{\prime} \mid \tau_i, \mathcal{T}_i\right)}{\mathbb{P}^{\prime}\left(s_{\tau_i+2 N}=s^{\prime} \mid \tau_i, \mathcal{T}_i\right)}=1+\frac{\mathbb{P}\left(s_{\tau_i+2 N}=s^{\prime} \mid \tau_i, \mathcal{T}_i\right)-\mu^\pi\left(s^{\prime}\right)}{\mu^\pi\left(s^{\prime}\right)} \leq 1+\frac{2}{T^4 \mu^\pi\left(s^{\prime}\right)} \leq 1+\frac{1}{T^3} \quad \text { for all } s^{\prime},
$$
であり，さらに

$$
\frac{\mathbb{P}(X)}{\mathbb{P}^{\prime}(X)} \leq\left(1+\frac{1}{T^3}\right)^M \leq e^{\frac{M}{T^3}} \leq e^{\frac{1}{T^2}} \leq 1+\frac{2}{T^2}
$$

も成り立ちます．よって，

$$
\frac{\mathbb{E}\left[\widehat{\beta}_k(s, a)\right]}{\mathbb{E}^{\prime}\left[\widehat{\beta}_k(s, a)\right]} \leq 1+\frac{2}{T^2}
$$

が成り立ちます．

---

これで$\widehat{\beta}_k$と$\beta^{\pi_k}$が絶対値について近いことが言えます．あと２乗誤差について近いことも言えますが，だいたい同じなので省略します